# Hyperparameters

In [1]:
hparam = {
    'epoch': 10,
    'hid_dim': 10,
    'gru_layers': 1,
    'lr': 0.0001,
    'report': 1
}

# Configure Dataset

In [2]:
from os import listdir
from os.path import exists
from itertools import chain
from random import shuffle
import pandas as pd
import pickle

# Load tensor file(pickle) list
dpath = './Dataset/matches_tensor/legacy/'
servers = listdir(dpath)
if not exists('./Dataset/match_list.pkl'):
    mat_lists = list(chain(*[listdir(dpath+server) for server in servers])) # total match lists
else:
    with open('./Dataset/match_list.pkl', 'rb') as file:
        mat_lists = pickle.load(file)

# Divide dataset into 3 (tr, val, te)
# shuffle(mat_lists)
# with open('./Dataset/match_list.pkl', 'wb') as file:
#     pickle.dump(mat_lists, file, pickle.HIGHEST_PROTOCOL)
tr_set = mat_lists[:190000]
val_set = mat_lists[190000:200000]
te_set = mat_lists[200000:]

# Load match results as dictionaris
rpath = './Dataset/matches_result/'
mat_results = {}
for server in servers:
    fname = rpath + f'{server}_results.csv'
    mat_results[server] = pd.read_csv(fname, index_col=0).to_dict('index')

# Model

In [3]:
from model import Model

M = Model(hparam['hid_dim'], hparam['gru_layers'], hparam['lr'])
# M.load_parameters('zero_h0_hiddim_1.pkl') # remove comment if you want load parameter file

# Train

In [ ]:
for epoch in range(hparam['epoch']):
    # training process
    for idx, tr_dat in enumerate(tr_set):
        for serv in ['kr', 'euw1', 'eun1', 'na1', 'jp1']:
            if serv in tr_dat:
                server = serv
                continue
        mat = tr_dat[:-4]

        # print training progress
        print(f'[Epoch {epoch+1:3d}] ({idx+1:6d}/{len(tr_set)}) processing {mat:<30s}', end='\r')

        with open(f'{dpath}{server}/{tr_dat}', 'rb') as feat_file:
            features = pickle.load(feat_file)
        winner = mat_results[server][mat]['win']

        # train
        M.train(features, winner)
    
    # validation process
    if (epoch+1)%hparam['report'] == 0: # execute validation in {hparam['report']} epoch interval
        M.reset_result()
        for idx, val_dat in enumerate(val_set):
            for serv in ['kr', 'euw1', 'eun1', 'na1', 'jp1']:
                if serv in val_dat:
                    server = serv
                    continue
            mat = val_dat[:-4]

            # print validation progress
            print(f'[Validation for epoch {epoch+1:3d}] ({idx+1:6d}/{len(val_set)}) processing {mat:<30s}', end='\r') 

            with open(f'{dpath}{server}/{val_dat}', 'rb') as feat_file:
                features = pickle.load(feat_file)
            winner = mat_results[server][mat]['win']
            
            # validate
            M.validate(features, winner)

        acc, pre, rec, f1, loss = M.get_result()
        print(f'[Validation for epoch {epoch+1:3d}] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')
        M.save_parameters('normal.pkl')

# Test

In [4]:
from torch import cuda

cuda.empty_cache()
M.reset_result()
for idx, te_dat in enumerate(te_set):
    save_path = './experiment/matches_scores/normal/'
    for serv in ['kr', 'euw1', 'eun1', 'na1', 'jp1']:
        if serv in te_dat:
            server = serv
            break
    mat = te_dat[:-4]

    # print training progress
    print(f'[Testing] ({idx+1:6d}/{len(te_set)}) processing {mat:<30s}', end='\r')

    with open(f'{dpath}{server}/{te_dat}', 'rb') as feat_file:
        features = pickle.load(feat_file)
    winner = mat_results[server][mat]['win']

    M.test(features, winner, save_path, mat)

acc, pre, rec, f1, loss = M.get_result()
print(f'[Test result] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')

[Test result] Accuracy: 0.9946 / Precision: 0.9951 / Recall: 0.9943 / F1 score: 0.9947 / loss: 20.7400
